In [ ]:
import pandas as pd


import numpy as np
import matplotlib.pyplot as plt
import os
import json
import pprint
from boto.s3.connection import S3Connection
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
import pylab as pl
import numpy as np


%matplotlib inline

# Limit rows disp# Limit rows displayed in notebook
pd.set_option('display.max_rows', 20)
pd.set_option('display.precision', 2)

import json
import pandas as pd
 
def convertBusiness(x):
    ob = json.loads(x)
    return {"business_id":ob["business_id"], "stars":ob["stars"], "city":ob["city"], "latitude":ob["latitude"], "longitude":ob["longitude"]}

def convertReviews(x):
    ''' Convert a json string to a flat python dictionary
    which can be passed into Pandas. '''
    ob = json.loads(x)
    return {"business_id":ob["business_id"], "stars":ob["stars"], "date":ob["date"]}

def createPandas(func, json_filename):
    return pd.DataFrame([func(line) for line in file(json_filename)])

print "Creating Business Pandas"
pd_business = createPandas(convertBusiness,'yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_business.json')
print "Creating Review Pandas"
pd_review = createPandas(convertReviews, 'yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.json')
print "Completed Creating Pandas"

pd_business_lasvegas = pd_business[pd_business["city"] == "Las Vegas"]
pd_review_lasvegas = pd.merge(pd_business_lasvegas, pd_review, left_on='business_id', right_on='business_id', how='inner')

''' K Means with results '''

from pylab import plot,show
from numpy import vstack,array
from numpy.random import rand
from scipy.cluster.vq import kmeans,kmeans2, vq

business_lasvegas_latlong = pd_business_lasvegas[["latitude", "longitude"]].values.tolist()
X = StandardScaler().fit_transform(business_lasvegas_latlong)

k = 8
centroids, labels = kmeans2(X,k)
idx,_ = vq(X,centroids)
colors = pl.cm.Spectral(np.linspace(0, 1, k))
for i in range(k):
    plot(X[idx==i,0],X[idx==i,1], 'o', markerfacecolor=colors[i])
plot(centroids[:,0],centroids[:,1],'sg',markersize=8)
show()

# Adding column Cluster indicating cluser number
pd_business_lasvegas['cluster'] = pd.Series(labels, index=pd_business_lasvegas.index)

pd_review_lasvegas = pd.merge(pd_business_lasvegas, pd_review, left_on='business_id', right_on='business_id', how='inner')
gb = pd_review_lasvegas.groupby(['business_id'])
gb = gb.agg({'date' : np.min})
gb["new"] = (gb["date"] >= "2011-01-01")
pd_review_lasvegas = pd_review_lasvegas.set_index('business_id')
pd_lasvegas = pd.merge(pd_review_lasvegas, gb, left_index=True, right_index=True, how='inner')
#print list(pd_lasvegas.columns.values)

#now on we consider data in only cluster 0

pd_lasvegas_cluster0 = pd_lasvegas[pd_lasvegas["cluster"] == 0] 
pd_lasvegas_new = pd_lasvegas_cluster0[pd_lasvegas_cluster0["new"] == True]
pd_lasvegas_old = pd_lasvegas_cluster0[pd_lasvegas_cluster0["new"] == False]


Creating Business Pandas
Creating Review Pandas

In [9]:
pd_lasvegas.columns.values
pd_lasvegas.pop('latitude')
pd_lasvegas.pop('longitude')
pd_lasvegas.pop('stars_x')
pd_lasvegas.rename(columns={'stars_y': 'rating'}, inplace=True)
pd_lasvegas.rename(columns={'date_y': 'start_date'}, inplace=True)
pd_lasvegas.rename(columns={'date_x': 'review_date'}, inplace=True)
pd_lasvegas.columns.values

KeyError: 'latitude'

In [4]:
pd_lasvegas.save("Vegas_data.pkl")
pd_lasvegas=pd.load('Vegas_data.pkl')

C:\Anaconda\lib\site-packages\pandas\core\common.py:3317: FutureWarning: load is deprecated, use read_pickle
  warnings.warn("load is deprecated, use read_pickle", FutureWarning)


In [6]:
print pd_lasvegas.columns.values

['city' 'latitude' 'longitude' 'stars_x' 'cluster' 'date_x' 'stars_y'
 'date_y' 'new']
